In [5]:
import os
import json
import pandas as pd
from PyPDF2 import PdfReader
import traceback

In [6]:
import sys
print(sys.executable)

c:\Users\PMLS\envs\myenv\python.exe


In [7]:

from dotenv import load_dotenv
import os

# Load variables from .env
load_dotenv()

print("API key loaded:", os.getenv("OPENAI_API_KEY")[:10], "...")

API key loaded: sk-proj-lw ...


In [8]:
from langchain.chat_models import ChatOpenAI,openai
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

In [13]:
key=os.getenv("OPENAI_API_KEY")

In [14]:
import os
llm=ChatOpenAI(
    model="gpt-4o-mini-2024-07-18",
    api_key=key,
    max_tokens=2000  
)



In [15]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate    
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2



In [16]:
TEMPLATE="""""Text: {text}" \
"You are an expert in creating engaging and challenging multiple choice questions (MCQs) for educational purposes.\
"Generate {number} multiple choice questions (MCQs) in the subject of {subject} in tone of {tone}.\" \
"Each question should have 4 options (A, B, C, D) with one correct answer.\" \
"Provide the output in the following JSON format: {response_json}." \
"Ensure the questions are clear, concise, and relevant to the provided text. Questions should not be repeated" \" \
"Ensure to make {number}    MCQs.\" """

In [17]:
response_json={
    "1": {
        "question": "string",
        "options": {
            "A": "string",
            "B": "string",
            "C": "string",
            "D": "string"
        },
        " correct answer": "string"
    },
    "2": 
         {
        "question": "string",
        "options": {
            "A": "string",
            "B": "string",
            "C": "string",
            "D": "string"
        },
        " correct answer": "string"
    },
    "3": {
        "question": "string",
        "options": {
            "A": "string",
            "B": "string",
            "C": "string",
            "D": "string"
        },
        " correct answer": "string"
    }
}

In [18]:
quiz_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE)

In [19]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_prompt,output_key="quiz",verbose=True)

C:\Users\PMLS\AppData\Local\Temp\ipykernel_7616\2504320390.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  quiz_chain=LLMChain(llm=llm,prompt=quiz_prompt,output_key="quiz",verbose=True)


In [20]:
TEMPLATE2="""You are an english expert. Giver multiple choice questions (MCQs) for {subject} \" \
"You need to evaluate the complexity of the questions " \" \
"Update the quiz questions which needs to be simplified or made complex based on the complexity level \" \" \
"Quiz mcqs: {quiz} \" 
you can check from above quiz mcqs and update the questions """

In [21]:
quiz_evaluate_prompt=PromptTemplate(
    input_variables=["quiz","subject"],
    template=TEMPLATE2)

In [22]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluate_prompt,output_key="reviewed_quiz",verbose=True)

In [23]:
generate_evaluate_chain=SequentialChain(
    chains=[quiz_chain,review_chain],
    input_variables=["text","number","subject","tone","response_json"],
    output_variables=["quiz","reviewed_quiz"],
    verbose=True)

In [24]:
file_path = 'D:\MCQ-GenAI\data.txt'

In [25]:
with open(file_path, 'r') as file:
    text = file.read()
    

In [26]:
print(text)


The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.
 The synonym self-teaching computers was also used in this time period.

The earliest machine learning program was introduced in the 1950s when Arthur Samuel invented a computer program that calculated the winning chance in checkers for each side, but the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[10] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[11] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[10] Other researchers who have studied human cognitive systems contributed to the 

In [27]:
json.dumps(response_json)

'{"1": {"question": "string", "options": {"A": "string", "B": "string", "C": "string", "D": "string"}, " correct answer": "string"}, "2": {"question": "string", "options": {"A": "string", "B": "string", "C": "string", "D": "string"}, " correct answer": "string"}, "3": {"question": "string", "options": {"A": "string", "B": "string", "C": "string", "D": "string"}, " correct answer": "string"}}'

In [28]:
number=5
subject="Machine Learning"
tone="simple"

In [29]:
#token usage tracking in langchain
from langchain_community.callbacks import get_openai_callback

with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
        "text":text,
        "number":number,
        'subject':subject,
        'tone':tone,
        "response_json":json.dumps(response_json)
    })

C:\Users\PMLS\AppData\Local\Temp\ipykernel_7616\3371434376.py:5: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response=generate_evaluate_chain(
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:
""Text: The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.
 The synonym self-teaching computers was also used in this time period.

The earliest machine learning program was introduced in the 1950s when Arthur Samuel invented a computer program that calculated the winning chance in checkers for each side, but the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[10] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[11] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[10] Other researchers who have studied human cogn

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:
You are an english expert. Giver multiple choice questions (MCQs) for Machine Learning " "You need to evaluate the complexity of the questions " " "Update the quiz questions which needs to be simplified or made complex based on the complexity level " " "Quiz mcqs: ```json
{
    "1": {
        "question": "Who coined the term 'machine learning'?",
        "options": {
            "A": "Donald Hebb",
            "B": "Tom M. Mitchell",
            "C": "Arthur Samuel",
            "D": "Walter Pitts"
        },
        "correct answer": "C"
    },
    "2": {
        "question": "What type of learning did the Cybertron machine use?",
        "options": {
            "A": "Supervised Learning",
            "B": "Unsupervised Learning",
            "C": "Reinforcement Learning",
            "D": "Deep Learning"
        },
        "correct answer": "C"
    },
    "3": {
        "question": "Which algorithm type focuses on classification and regress

In [30]:
print(cb.total_tokens)
print(cb.prompt_tokens)
print(cb.total_cost)
print(cb.completion_tokens)

2325
1352
0.0007865999999999999
973


In [31]:
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.\n The synonym self-teaching computers was also used in this time period.\n\nThe earliest machine learning program was introduced in the 1950s when Arthur Samuel invented a computer program that calculated the winning chance in checkers for each side, but the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[10] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[11] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[10] Other researchers who have studied human cognitive systems contr

In [38]:
quiz=response.get("quiz")

In [40]:
import re

quiz_clean = re.sub(r"^```json|```$", "", quiz, flags=re.MULTILINE).strip()

quiz_dict = json.loads(quiz_clean)
print(quiz_dict)

{'1': {'question': "Who coined the term 'machine learning'?", 'options': {'A': 'Donald Hebb', 'B': 'Tom M. Mitchell', 'C': 'Arthur Samuel', 'D': 'Walter Pitts'}, 'correct answer': 'C'}, '2': {'question': 'What type of learning did the Cybertron machine use?', 'options': {'A': 'Supervised Learning', 'B': 'Unsupervised Learning', 'C': 'Reinforcement Learning', 'D': 'Deep Learning'}, 'correct answer': 'C'}, '3': {'question': 'Which algorithm type focuses on classification and regression?', 'options': {'A': 'Unsupervised Learning Algorithms', 'B': 'Reinforcement Learning Algorithms', 'C': 'Supervised Learning Algorithms', 'D': 'Neural Networks'}, 'correct answer': 'C'}, '4': {'question': 'What book did Tom M. Mitchell provide a formal definition for machine learning?', 'options': {'A': 'The Organization of Behavior', 'B': 'Learning Machines', 'C': 'Computing Machinery and Intelligence', 'D': 'Patterns in Machine Learning'}, 'correct answer': 'B'}, '5': {'question': 'What is the objective o